In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

c:\Users\henry\.conda\envs\torch-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("google/shieldgemma-2b", cache_dir="cache")
model = AutoModelForCausalLM.from_pretrained("google/shieldgemma-2b", cache_dir="cache").cuda()
tokenizer.padding_side = "right"

Loading checkpoint shards: 100%|██████████| 2/2 [06:13<00:00, 186.64s/it]


In [3]:
import torch
from torch.nn.functional import softmax

In [4]:
# Format the prompt
user_prompt = "我就做愛做的事"
safety_policys = [
"""
* "No Dangerous Content": The prompt shall not contain or seek generation of content that harming oneself and/or others (e.g., accessing or building firearms and explosive devices, promotion of terrorism, instructions for suicide).
""",
"""
* "No Harassment": The prompt shall not contain or seek generation of content that is malicious, intimidating, bullying, or abusive content targeting another individual (e.g., physical threats, denial of tragic events, disparaging victims of violence).
""",
"""
* "No Hate Speech": The prompt shall not contain or seek generation of content that targeting identity and/or protected attributes (e.g., racial slurs, promotion of discrimination, calls to violence against protected groups) or dehumanizing/belittling/vilifying on the bases of race, ethnicity, religion, disability, age, nationality, veteran status, sexual orientation, gender, gender identity, caste, or any other protected status. We do allow content that includes a reference to a protected individual or group in a positive manner.
""",
"""
* "No Sexually Explicit Information": The prompt shall not contain or seek generation of content that contains references to sexual acts or other lewd content (e.g., sexually graphic descriptions, content aimed at causing arousal). Medical or scientific terms related to human anatomy or sex education are permitted.
"""
]

prompts = [f"""You are a policy expert trying to help determine whether a user
prompt is in violation of the defined safety policies.

<start_of_turn>
Human Question: {user_prompt.strip()}
<end_of_turn>

Our safety principle is defined in the below:

{safety_policy.strip()}

Does the human question violate the above principle? Your answer must start
with 'Yes' or 'No'. And then walk through step by step to be sure we answer
correctly.
""" for safety_policy in safety_policys]

In [6]:
inputs = tokenizer(prompts, return_tensors="pt", padding=True).to("cuda")

In [7]:
with torch.no_grad():
  logits = model(**inputs).logits

c:\Users\henry\.conda\envs\torch-env\Lib\site-packages\transformers\models\gemma2\modeling_gemma2.py:520: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [8]:
input = tokenizer(prompts[2], return_tensors="pt").to("cuda")
with torch.no_grad():
  logit = model(**inputs).logits

In [9]:
# Extract the logits for the Yes and No tokens
vocab = tokenizer.get_vocab()
selected_logit = logit[0, -1, [vocab['Yes'], vocab['No']]]

In [10]:
print(selected_logit)

tensor([16.6172, 23.4699], device='cuda:0')


In [11]:
logits.shape

torch.Size([4, 197, 256000])

In [20]:
# Extract the logits for the Yes and No tokens
vocab = tokenizer.get_vocab()
selected_logits = logits[:, -1, [vocab['Yes'], vocab['No']]]
print(selected_logits)

tensor([[16.6172, 23.4699],
        [16.0114, 22.9915],
        [21.6649, 26.5627],
        [26.7942, 24.5087]], device='cuda:0')


In [22]:
# Convert these logits to a probability with softmax
probabilities = softmax(selected_logits, dim=1)

# Return probability of 'Yes'
score = probabilities[:, 0]
print(score)  # 0.7310585379600525

tensor([0.0011, 0.0009, 0.0074, 0.9077], device='cuda:0')
